In [ ]:
import missingno as msno
from pandas_profiling import ProfileReport
import functions as fct
import warnings
import cufflinks as cf
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.offline as py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
color = sns.color_palette()
py.init_notebook_mode(connected=True)
init_notebook_mode(connected=True)
offline.init_notebook_mode()
cf.go_offline()
warnings.simplefilter(action='ignore', category=FutureWarning)



  # Chargement des données

In [ ]:
application_train = pd.read_csv('../01_DATA/application_train.csv')
POS_CASH_balance = pd.read_csv('../01_DATA/POS_CASH_balance.csv')
bureau_balance = pd.read_csv('../01_DATA/bureau_balance.csv')
previous_application = pd.read_csv('../01_DATA/previous_application.csv')
installments_payments = pd.read_csv('../01_DATA/installments_payments.csv')
credit_card_balance = pd.read_csv('../01_DATA/credit_card_balance.csv')
bureau = pd.read_csv('../01_DATA/bureau.csv')
application_test = pd.read_csv('../01_DATA/application_test.csv')



In [ ]:
print('Size of application_train data', application_train.shape)
print('Size of POS_CASH_balance data', POS_CASH_balance.shape)
print('Size of bureau_balance data', bureau_balance.shape)
print('Size of previous_application data', previous_application.shape)
print('Size of installments_payments data', installments_payments.shape)
print('Size of credit_card_balance data', credit_card_balance.shape)
print('Size of bureau data', bureau.shape)



In [ ]:
# profile = ProfileReport(application_train)
# profile.to_file(output_file='rapport_application_train.html')



  # Aperçu des données disponibles

In [ ]:
application_train.info(verbose=1, show_counts=1)



In [ ]:
POS_CASH_balance.info(verbose=1, show_counts=1)



In [ ]:
bureau_balance.info(verbose=1, show_counts=1)



In [ ]:
previous_application.info(verbose=1, show_counts=1)



In [ ]:
installments_payments.info(verbose=1, show_counts=1)



In [ ]:
credit_card_balance.info(verbose=1, show_counts=1)



In [ ]:
bureau.info(verbose=1, show_counts=1)



  # Target

In [ ]:
application_train['TARGET'].value_counts(dropna=False)



In [ ]:
application_train.TARGET.plot.hist()



In [ ]:
temp = application_train["TARGET"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels',
         values='values', title='Loan Repayed or not')



  Les classes cibles sont déséquilibrées : 92% des prêts sont remboursés.

  # Valeurs manquantes

In [ ]:
# Function to calculate missing values by column# Funct


def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns



  Application_train

In [ ]:
fct.pourcent_NA_df(application_train)
msno.matrix(application_train)
# Missing values statistics
missing_values = missing_values_table(application_train)
missing_values.head(20)



  POS_CASH_balance

In [ ]:
fct.pourcent_NA_df(POS_CASH_balance)
msno.matrix(POS_CASH_balance)
# Missing values statistics
missing_values = missing_values_table(POS_CASH_balance)
missing_values.head(20)



  Bureau_balance

In [ ]:
fct.pourcent_NA_df(bureau_balance)
msno.matrix(bureau_balance)
# Missing values statistics
missing_values = missing_values_table(bureau_balance)
missing_values.head(20)



  Previous_application

In [ ]:
fct.pourcent_NA_df(previous_application)
msno.matrix(previous_application)
# Missing values statistics
missing_values = missing_values_table(previous_application)
missing_values.head(20)



  Installment_payment

In [ ]:
fct.pourcent_NA_df(installments_payments)
msno.matrix(installments_payments)
# Missing values statistics
missing_values = missing_values_table(installments_payments)
missing_values.head(20)



  Credit_balance

In [ ]:
fct.pourcent_NA_df(credit_card_balance)
msno.matrix(credit_card_balance)
# Missing values statistics
missing_values = missing_values_table(credit_card_balance)
missing_values.head(20)



  bureau

In [ ]:
fct.pourcent_NA_df(bureau)
msno.matrix(bureau)
# Missing values statistics
missing_values = missing_values_table(bureau)
missing_values.head(20)



  Pour le moment on ne peut pas savoir si ces colonnes sont importantes pour la suite de l'étude. Je les garde donc pour le moment.

  # Type de colonnes

In [ ]:
# Number of each type of column
application_train.dtypes.value_counts()



In [ ]:
# nombre d'entrées différentes pour les colonnes 'object'
application_train.select_dtypes('object').apply(pd.Series.nunique, axis=0)



  La plupart des colonnes catégorielles ne présentent pas beaucoup de valeurs différentes (cardinalité faible, sauf pour les colonnes occupation_type et organisation_type).

  Les colonnes présentant 2 valeurs pourront être transformées avec un label_encoder (4 colonnes), les autres pourront être transformées via un one hot encoder.

  # Data exploration
  ## données catégorielles

In [ ]:
col_obj = list(application_train.select_dtypes('object').columns)
for col in col_obj:
    print('-'*10)
    print(col)
    print(application_train[col].value_counts(dropna=False))


 code gender : 4 code XNA qui doivent correspondrent à NaN

 ORGANIZATION_TYPE : 55353 code XNA

 ## variables DAYS
  ### days_birth

In [ ]:
(application_train['DAYS_BIRTH'] / -365).describe()



  Pas d'anomalie sur l'age des personnes ayant candidaté pour un pret
  ### Days_employed

In [ ]:
(application_train.DAYS_EMPLOYED / -365).describe()



  Probleme avec la valeur min qui représente 1000 ans...

In [ ]:
application_train.DAYS_EMPLOYED.plot.hist(title='Days employment histogram')
plt.xlabel('Days Employment')



In [ ]:
application_train.DAYS_EMPLOYED.value_counts()



In [ ]:
temp = application_train[(application_train.DAYS_EMPLOYED < -350)
                         & (application_train.DAYS_EMPLOYED > -380)]

sns.histplot(data=temp, x='DAYS_EMPLOYED', bins=28)



  Cette anomalie est présente pour 55374 clients et prend systématiquement la même valeur 365243 (qui ressemble au nombre de jours dans une année, en oubliant la virgule : 365.243). Regardons plus en détail la répartition de cette anomalie en fonction de la target

In [ ]:
temp = application_train[(application_train.DAYS_EMPLOYED < -300)
                         & (application_train.DAYS_EMPLOYED > -400)]
plt.figure(figsize=(20, 7))
sns.histplot(data=temp, x='DAYS_EMPLOYED', bins=98)



In [ ]:
masque = (application_train.DAYS_EMPLOYED == 365243)
anomalie = application_train[masque]
non_anomalie = application_train[~masque]
refus_anomalies = 100 * anomalie.TARGET.mean()
refus_non_anomalies = 100 * non_anomalie.TARGET.mean()
print('{:.2f} % des clients ayant un pb sur la durée de'.format(refus_anomalies),
      'travail présentent un refus de prêt')
print('{:.2f} % des clients ayant une durée de'.format(refus_non_anomalies),
      'travail normale présentent un refus de prêt')



  Les clients présentant une anomalie sur leur durée d'emplois présentent moins de refus de prêt.

  Comme la valeur de cette anomalie est constante (days_employed = 365243), je vais garder l'information de cete anomalie au cas où elle corresponde à une autre information.

In [ ]:
# creation d'une colonne pour stocker l'info anomalie / non  anomalie
application_train['DAYS_EMPLOYED_ANOM'] = masque

# remplacement des valeurs anormales par np.NaN
application_train.DAYS_EMPLOYED.replace({365243: np.nan}, inplace=True)

application_train.DAYS_EMPLOYED.plot.hist(title='Days employment histogram')
plt.xlabel('Days Employment')



  La distribution est plus cohérente avec ce qui est attendu.

  ### Days_id_publish

In [ ]:
(application_train.DAYS_ID_PUBLISH / -365).describe()



In [ ]:
application_train.DAYS_ID_PUBLISH.plot.hist(
    title='Days ID published histogram')
plt.xlabel('Days ID published')



  pas de soucis

  ### Days_last_phone_change

In [ ]:
(application_train.DAYS_LAST_PHONE_CHANGE / -365).describe()



In [ ]:
application_train.DAYS_LAST_PHONE_CHANGE.plot.hist(
    title='Days last phone change histogram')
plt.xlabel('Days last phone change')



  Pas de soucis

  ### Days_registration

In [ ]:
(application_train.DAYS_REGISTRATION / -365).describe()



In [ ]:
application_train.DAYS_REGISTRATION.plot.hist(
    title='Days registration histogram')
plt.xlabel('Days registration')



  Quelques outliers probables au delà de 20000 jours (soit 54 ans)

In [ ]:
masque = (application_train.DAYS_REGISTRATION < -20000)
print("Il y a {} lignes présentant une valeur de".format(masque.sum()),
      "days_registration < -20 000")
anomalie = application_train[masque]
non_anomalie = application_train[~masque]
refus_anomalies = 100 * anomalie.TARGET.mean()
refus_non_anomalies = 100 * non_anomalie.TARGET.mean()
print('{:.2f} % des clients ayant un pb sur days_registration'.format(refus_anomalies),
      'présentent un refus de prêt')
print("{:.2f} % des clients n'ayant pas de pb sur days_registration".format(refus_non_anomalies),
      'présentent un refus de prêt')



In [ ]:
application_train[masque]


  Les clients présentant une anomalie sur days_registration présentent moins de refus de prêt.

  Je vais garder l'information de cete anomalie au cas où elle corresponde à une autre information.

In [ ]:
# creation d'une colonne pour stocker l'info anomalie / non  anomalie
application_train['DAYS_REGISTRATION_ANOM'] = masque

# remplacement des valeurs anormales par np.NaN
application_train[masque].DAYS_REGISTRATION = np.nan

application_train.DAYS_REGISTRATION.plot.hist(
    title='Days registration histogram')
plt.xlabel('Days registration')



  ## Distribution des variables quantitatives
  ### AMT_CREDIT

In [ ]:
plt.figure(figsize=(12, 5))
ax = sns.histplot(application_train["AMT_CREDIT"])
plt.title("Distribution of AMT_CREDIT")



  ### AMT_INCOME_TOTAL

In [ ]:
plt.figure(figsize=(12, 5))

ax = sns.distplot(application_train["AMT_INCOME_TOTAL"].dropna())
plt.title("Distribution of AMT_INCOME_TOTAL")



  ### AMT_GOODS_PRICE

In [ ]:
plt.figure(figsize=(12, 5))

ax = sns.histplot(application_train["AMT_GOODS_PRICE"].dropna())
plt.title("Distribution of AMT_GOODS_PRICE")



  ### NAME_TYPE_SUITE : personne qui accompagne le demandeur

In [ ]:
temp = application_train["NAME_TYPE_SUITE"].value_counts()
#print("Total number of states : ",len(temp))
trace = go.Bar(
    x=temp.index,
    y=(temp / temp.sum())*100,
)
data = [trace]
layout = go.Layout(
    title="Who accompanied client when applying for the  application in % ",
    xaxis=dict(
        title='Name of type of the Suite',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of Name of type of the Suite in %',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')



  ### NAME_CONTRACT_TYPE : type de prêt

In [ ]:
temp = application_train["NAME_CONTRACT_TYPE"].value_counts()
fig = {
    "data": [
      {
          "values": temp.values,
          "labels": temp.index,
          "domain": {"x": [0, .48]},
          # "name": "Types of Loans",
          # "hoverinfo":"label+percent+name",
          "hole": .7,
          "type": "pie"
      },

    ],
    "layout": {
        "title": "Types of loan",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Loan Types",
                "x": 0.17,
                "y": 0.5
            }

        ]
    }
}
iplot(fig, filename='donut')



  ### Possession de voiture / maison

In [ ]:
temp1 = application_train["FLAG_OWN_CAR"].value_counts()
temp2 = application_train["FLAG_OWN_REALTY"].value_counts()

fig = {
    "data": [
      {
          "values": temp1.values,
          "labels": temp1.index,
          "domain": {"x": [0, .48]},
          "name": "Own Car",
          "hoverinfo":"label+percent+name",
          "hole": .6,
          "type": "pie"
      },
        {
          "values": temp2.values,
          "labels": temp2.index,
          # "text":"Own Realty",
          "textposition": "inside",
          "domain": {"x": [.52, 1]},
          "name": "Own Reality",
          "hoverinfo":"label+percent+name",
          "hole": .6,
          "type": "pie"
      }],
    "layout": {
        "title": "What applicant own",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Own Car",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Own Realty",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}
iplot(fig, filename='donut')



  ### Source de revenu

In [ ]:
temp = application_train["NAME_INCOME_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Income sources of Applicant\'s', hole=0.5)



  ### Statut familial

In [ ]:
temp = application_train["NAME_FAMILY_STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Family Status of Applicant\'s', hole=0.5)



  ### Profession

In [ ]:
temp = application_train["OCCUPATION_TYPE"].value_counts()
temp.iplot(kind='bar', xTitle='Occupation', yTitle="Count",
           title='Occupation of Applicant\'s who applied for loan', color='green')



  ### niveau d'études

In [ ]:
temp = application_train["NAME_EDUCATION_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Education of Applicant\'s', hole=0.5)



  ### Lieu de résidence

In [ ]:
temp = application_train["NAME_HOUSING_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Type of House', hole=0.5)



  ### Secteur d'emplois

In [ ]:
temp = application_train["ORGANIZATION_TYPE"].value_counts()
temp.iplot(kind='bar', xTitle='Organization Name', yTitle="Count",
           title='Types of Organizations who applied for loan ', color='red')



  ## Exploration en fonction de la target
  ### Revenus

In [ ]:
temp = application_train["NAME_INCOME_TYPE"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["NAME_INCOME_TYPE"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["NAME_INCOME_TYPE"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction de la source de revenu",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  ### Statut familial

In [ ]:
temp = application_train["NAME_FAMILY_STATUS"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["NAME_FAMILY_STATUS"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["NAME_FAMILY_STATUS"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
# print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction du statut familial",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  ### Profession

In [ ]:
temp = application_train["OCCUPATION_TYPE"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["OCCUPATION_TYPE"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["OCCUPATION_TYPE"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
# print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction de la profession",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  ### Niveau d'étude

In [ ]:
temp = application_train["NAME_EDUCATION_TYPE"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["NAME_EDUCATION_TYPE"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["NAME_EDUCATION_TYPE"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
# print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction de la profession",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  ### lieu de résidence

In [ ]:
temp = application_train["NAME_HOUSING_TYPE"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["NAME_HOUSING_TYPE"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["NAME_HOUSING_TYPE"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
# print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction du lieu d'habitation",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  ### Secteur d'emplois

In [ ]:
temp = application_train["ORGANIZATION_TYPE"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["ORGANIZATION_TYPE"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["ORGANIZATION_TYPE"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
# print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction du secteur d'emplois",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  ### Accompagnant du demandeur de prêt

In [ ]:
temp = application_train["NAME_TYPE_SUITE"].value_counts()
# print(temp.values)
temp_y0 = []
temp_y1 = []
ratio = []
i = 0
for val in temp.index:
    temp_y1.append(np.sum(
        application_train["TARGET"][application_train["NAME_TYPE_SUITE"] == val] == 1))
    temp_y0.append(np.sum(
        application_train["TARGET"][application_train["NAME_TYPE_SUITE"] == val] == 0))
    ratio.append(100 * (temp_y1[i] / (temp_y1[i] + temp_y0[i])))
    i += 1
# print(temp_y1)
# print(temp_y0)
# print(ratio)
trace1 = go.Bar(
    x=temp.index,
    y=ratio,
    # name='YES'
)


data = [trace1]
layout = go.Layout(
    title="Pourcentage de prêts refusés en fonction de l'accompagnant du demandeur",
    # barmode='stack',
    width=1000,
    xaxis=dict(
        title='Income source',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Prêts refusés (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)



  # Previous application : data exploration

In [ ]:
nb_pret = len(previous_application.SK_ID_CURR.unique())
nb_pret_tot = len(application_train.SK_ID_CURR.unique()) + \
    len(application_test.SK_ID_CURR.unique())
print("{} pret en cours on été fait par des personnes ayant".format(nb_pret),
      "déjà fait une demande de prêt par le passé",
      " Soit {:.2f} % des prêts en cours".format(100*nb_pret/nb_pret_tot))



  ## Type de contrats précédemment contractés

In [ ]:
temp = previous_application["NAME_CONTRACT_TYPE"].value_counts()
fig = {
    "data": [
      {
          "values": temp.values,
          "labels": temp.index,
          "domain": {"x": [0, .48]},
          # "name": "Types of Loans",
          # "hoverinfo":"label+percent+name",
          "hole": .7,
          "type": "pie"
      },

    ],
    "layout": {
        "title": "Contract product type of previous application",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Contract product type",
                "x": 0.12,
                "y": 0.5
            }

        ]
    }
}
iplot(fig, filename='donut')



  ## Utilisation du prêt

In [ ]:
temp = previous_application["NAME_CASH_LOAN_PURPOSE"].value_counts()
#print("Total number of states : ",len(temp))
trace = go.Bar(
    x=temp.index,
    y=(temp / temp.sum())*100,
    marker=dict(
        color=(temp / temp.sum())*100,
        colorscale='Blues',
        reversescale=True
    ),
)
data = [trace]
layout = go.Layout(
    title="Purpose of cash loan in previous application in % ",
    xaxis=dict(
        title='Purpose of cash loan',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count in %',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)



  Information globalement peu communiquée (hyp : XAP et XNA correspondent à NaN)

  ## Statut des demandes précédentes

In [ ]:
temp = previous_application["NAME_CONTRACT_STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Contract approved or not', hole=0.5)



  ## Type de rembourement choisi par le demandeur

In [ ]:
temp = previous_application["NAME_PAYMENT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Payment method that client choose to pay for the previous application', hole=0.7, colors=['#75e575', '#ea7c96', ])



  ## Raisons du refus

In [ ]:
temp = previous_application["CODE_REJECT_REASON"].value_counts()
temp.iplot(kind='bar', xTitle='Reason', yTitle="Count",
           title='Top reasons of previous application rejection(Wriiten in some codes)', colors=['#75e575'])



  ## demande d'assurance pour un prêt précédent

In [ ]:
temp = previous_application["NFLAG_INSURED_ON_APPROVAL"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                   })
df.iplot(kind='pie', labels='labels', values='values',
         title='Did the client requested insurance during the previous application(YES : 1, NO : 0)', hole=0.7, colors=['#ea7c96', '#75e575', ])



  # Correlation de pearson

In [ ]:
# calcul de la matrice de corrélation et visualisation sous forme de heatmap
corr = application_train.corr()
# pour masquer la partie supérieure droite de la matrice de corrélation
mask = np.triu(np.ones_like(corr, dtype=bool))

# graph
fig = plt.figure(figsize=(25, 22))
sns.heatmap(data=corr, mask=mask, cmap='coolwarm')
plt.title("Coefficients de corrélation de Pearson")
plt.show()



  Des variables semblent corrélées entre elles (variables qui correspondent à des valeurs midi, mini, mode, avg de mêmes variables)

In [ ]:
# Find correlations with the target and sort
correlations = application_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))



  Les correlations positive avec la target sont très faibles (max = 0.08). Par contre les corrélation négatives présentent des valeurs un peu plus élevées (-0.18)
  ## Days_BIRTH vs TARGET

In [ ]:
# Age information into a separate dataframe
age_data = application_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / -365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(
    age_data['YEARS_BIRTH'], bins=np.linspace(20, 70, num=11))
age_data.head(10)



In [ ]:
# Group by the bin and calculate averages
age_groups = age_data.groupby('YEARS_BINNED').mean()
age_groups



In [ ]:
plt.figure(figsize=(8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation=75)
plt.xlabel('Age Group (years)')
plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group')



  plus un demandeur est jeune plus il risque de ne pas réussir à rembourser son prêt
  ## EXT_SOURCE vs TARGET

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
ext_data = application_train[['TARGET', 'EXT_SOURCE_1',
                              'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs



In [ ]:
plt.figure(figsize=(8, 6))

# Heatmap of correlations
sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r,
            vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap')



  Plus la valeur de ext_source augmente, plus le client est susceptible de rembourser le prêt

 # Nettoyage et création de features

In [ ]:
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [
        col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns,
                        dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


 ## Application_train

In [ ]:
# Remove 4 applications with XNA CODE_GENDER
print(application_train.shape)
application_train = application_train[application_train['CODE_GENDER'] != 'XNA']
print(application_train.shape)


In [ ]:
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'DAYS_EMPLOYED_ANOM', 'DAYS_REGISTRATION_ANOM']:
    print(bin_feature)
    print(application_train[bin_feature].unique())
    print(application_train[bin_feature].value_counts(dropna=False))
    application_train[bin_feature], uniques = pd.factorize(
        application_train[bin_feature])
    print(application_train[bin_feature].unique())
    print(application_train[bin_feature].value_counts(dropna=False))


In [ ]:
# Categorical features with One-Hot encode
application_train, cat_cols = one_hot_encoder(
    application_train, nan_as_category=False)

# Some simple new features (percentages)
application_train['DAYS_EMPLOYED_PERC'] = application_train['DAYS_EMPLOYED'] / \
    application_train['DAYS_BIRTH']
application_train['INCOME_CREDIT_PERC'] = application_train['AMT_INCOME_TOTAL'] / \
    application_train['AMT_CREDIT']
application_train['INCOME_PER_PERSON'] = application_train['AMT_INCOME_TOTAL'] / \
    application_train['CNT_FAM_MEMBERS']
application_train['ANNUITY_INCOME_PERC'] = application_train['AMT_ANNUITY'] / \
    application_train['AMT_INCOME_TOTAL']
application_train['PAYMENT_RATE'] = application_train['AMT_ANNUITY'] / \
    application_train['AMT_CREDIT']


 ## Bureau_balance & bureau

In [ ]:
nan_as_category = True
bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
bureau_balance, bureau_balance_cat = one_hot_encoder(
    bureau_balance, nan_as_category)
bureau.info()


In [ ]:
bureau_balance.info()


In [ ]:
# Bureau balance: Perform aggregations and merge with bureau.csv
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
for col in bureau_balance_cat:
    bb_aggregations[col] = ['mean']
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper()
                           for e in bb_agg.columns.tolist()])
bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
bureau


In [ ]:
bureau.info()


In [ ]:
# Bureau and bureau_balance numeric features
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum'],
    'MONTHS_BALANCE_MIN': ['min'],
    'MONTHS_BALANCE_MAX': ['max'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum']
}


In [ ]:
# Bureau and bureau_balance categorical features
cat_aggregations = {}
for cat in bureau_cat:
    cat_aggregations[cat] = ['mean']
for cat in bureau_balance_cat:
    cat_aggregations[cat + "_MEAN"] = ['mean']
bureau_agg = bureau.groupby('SK_ID_CURR').agg(
    {**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(
    ['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
bureau_agg


In [ ]:
# Bureau: Active credits - using only numerical aggregations
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
active_agg.columns = pd.Index(
    ['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')


In [ ]:
# Bureau: Closed credits - using only numerical aggregations
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(
    ['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')


 ## previous applications

In [ ]:
previous_application, cat_cols = one_hot_encoder(
    previous_application, nan_as_category=True)
# Days 365.243 values -> nan
previous_application['DAYS_FIRST_DRAWING'].replace(
    365243, np.nan, inplace=True)
previous_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_LAST_DUE_1ST_VERSION'].replace(
    365243, np.nan, inplace=True)
previous_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)


In [ ]:
# Add feature: value ask / value received percentage
previous_application['APP_CREDIT_PERC'] = previous_application['AMT_APPLICATION'] / \
    previous_application['AMT_CREDIT']


In [ ]:
# Previous applications numeric features
num_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
}


In [ ]:
# Previous applications categorical features
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']

prev_agg = previous_application.groupby('SK_ID_CURR').agg(
    {**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(
    ['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
prev_agg


In [ ]:
# Previous Applications: Approved Applications - only numerical features
approved = previous_application[previous_application['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(
    ['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
prev_agg


In [ ]:
# Previous Applications: Refused Applications - only numerical features
refused = previous_application[previous_application['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(
    ['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
prev_agg.info()


 ## POS_CASH_Balance

In [ ]:
POS_CASH_balance, cat_cols = one_hot_encoder(
    POS_CASH_balance, nan_as_category=True)


In [ ]:
# Features
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']

pos_agg = POS_CASH_balance.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(
    ['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])


In [ ]:
# Count pos cash accounts
pos_agg['POS_COUNT'] = POS_CASH_balance.groupby('SK_ID_CURR').size()
pos_agg.info()


 ## Installment_payments

In [ ]:
installments_payments, cat_cols = one_hot_encoder(
    installments_payments, nan_as_category=True)


In [ ]:
# Percentage and difference paid in each installment (amount paid and installment value)
installments_payments['PAYMENT_PERC'] = installments_payments['AMT_PAYMENT'] / \
    installments_payments['AMT_INSTALMENT']
installments_payments['PAYMENT_DIFF'] = installments_payments['AMT_INSTALMENT'] - \
    installments_payments['AMT_PAYMENT']
# Days past due and days before due (no negative values)
installments_payments['DPD'] = installments_payments['DAYS_ENTRY_PAYMENT'] - \
    installments_payments['DAYS_INSTALMENT']
installments_payments['DBD'] = installments_payments['DAYS_INSTALMENT'] - \
    installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['DPD'] = installments_payments['DPD'].apply(
    lambda x: x if x > 0 else 0)
installments_payments['DBD'] = installments_payments['DBD'].apply(
    lambda x: x if x > 0 else 0)


In [ ]:
# Features: Perform aggregations
aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']
ins_agg = installments_payments.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(
    ['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
ins_agg


In [ ]:
# Count installments accounts
ins_agg['INSTAL_COUNT'] = installments_payments.groupby('SK_ID_CURR').size()
ins_agg.info()


 ## credit_card_balance

In [ ]:
credit_card_balance, cat_cols = one_hot_encoder(
    credit_card_balance, nan_as_category=True)


In [ ]:
# General aggregations
credit_card_balance.drop(['SK_ID_PREV'], axis=1, inplace=True)
cc_agg = credit_card_balance.groupby('SK_ID_CURR').agg(
    ['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper()
                           for e in cc_agg.columns.tolist()])
cc_agg


In [ ]:
# Count credit card lines
cc_agg['CC_COUNT'] = credit_card_balance.groupby('SK_ID_CURR').size()
cc_agg.info()


 ## Concatenation des différents fichiers

In [ ]:
data = application_train.copy()
print(data.shape)
data = data.join(bureau_agg, how='left', on='SK_ID_CURR')
print(data.shape)
data = data.join(prev_agg, how='left', on='SK_ID_CURR')
print(data.shape)
data = data.join(pos_agg, how='left', on='SK_ID_CURR')
print(data.shape)
data = data.join(ins_agg, how='left', on='SK_ID_CURR')
print(data.shape)
data = data.join(cc_agg, how='left', on='SK_ID_CURR')
print(data.shape)


  # Export des données pour la modélisation

In [ ]:
data.to_csv('../01_DATA/data_for_modelling.csv')
